# Problem: Predicting Airplane Delays

The goals of this notebook are:
- Process and create a dataset from downloaded ZIP files
- Exploratory data analysis (EDA)
- Establish a baseline model and improve it

## Introduction to business scenario
You work for a travel booking website that is working to improve the customer experience for flights that were delayed. The company wants to create a feature to let customers know if the flight will be delayed due to weather when the customers are booking the flight to or from the busiest airports for domestic travel in the US. 

You are tasked with solving part of this problem by leveraging machine learning to identify whether the flight will be delayed due to weather. You have been given access to the a dataset of on-time performance of domestic flights operated by large air carriers. You can use this data to train a machine learning model to predict if the flight is going to be delayed for the busiest airports.

### Dataset
The provided dataset contains scheduled and actual departure and arrival times reported by certified US air carriers that account for at least 1 percent of domestic scheduled passenger revenues. The data was collected by the Office of Airline Information, Bureau of Transportation Statistics (BTS). The dataset contains date, time, origin, destination, airline, distance, and delay status of flights for flights between 2014 and 2018.
The data are in 60 compressed files, where each file contains a CSV for the flight details in a month for the five years (from 2014 - 2018). The data can be downloaded from this link: [https://ucstaff-my.sharepoint.com/:f:/g/personal/ibrahim_radwan_canberra_edu_au/Er0nVreXmihEmtMz5qC5kVIB81-ugSusExPYdcyQTglfLg?e=bNO312]. Please download the data files and place them on a relative path. Dataset(s) used in this assignment were compiled by the Office of Airline Information, Bureau of Transportation Statistics (BTS), Airline On-Time Performance Data, available with the following link: [https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ]. 

# Step 1: Prepare the environment 

Use one of the labs which we have practised on with the Amazon Sagemakers where you perform the following steps:
1. Start a lab.
2. Create a notebook instance and name it "oncloudproject".
3. Increase the used memory to 25 GB from the additional configurations.
4. Open Jupyter Lab and upload this notebook into it.
5. Upload the two combined CVS files (combined_csv_v1.csv and combined_csv_v2.csv), which you created in Part A of this project.

# Step 2: Build and evaluate simple models

Write code to perform the follwoing steps:
1. Split data into training, validation and testing sets (70% - 15% - 15%).
2. Use linear learner estimator to build a classifcation model.
3. Host the model on another instance
4. Perform batch transform to evaluate the model on testing data
5. Report the performance metrics that you see better test the model performance 

Note: You are required to perform the above steps on the two combined datasets separatey and to comments on the difference.

## Part 1: combined_csv_v1.csv

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data/combined_csv_v1.csv")
df.head(3)

,target,Distance,Quarter_2,Quarter_3,Quarter_4,Month_2,Month_3,Month_4,Month_5,Month_6,...,DepHourofDay_14,DepHourofDay_15,DepHourofDay_16,DepHourofDay_17,DepHourofDay_18,DepHourofDay_19,DepHourofDay_20,DepHourofDay_21,DepHourofDay_22,DepHourofDay_23
0,0.0,689.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,0.0,731.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0.0,1199.0,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


### Data Preprocessing

In [4]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(df, test_size=0.3)
validation, test = train_test_split(validation, test_size = 0.5)

In [5]:
train.to_csv("v1a/train.csv", index = False, header = False)
validation.to_csv("v1a/validation.csv", index = False, header = False)
test.to_csv("v1a/test.csv", index = False, header = False)

## Instantiating sagemaker session, getting default bucket and fetching the linear learner container

In [6]:
import sagemaker
import boto3
from sagemaker import image_uris

sess = sagemaker.Session()
bucket = sess.default_bucket()


region = boto3.Session().region_name
linear_learner_container = image_uris.retrieve("linear-learner", region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### Defining paths to Amazon S3 bucket

In [7]:
prefix = "flights_delay"
train_path = sess.upload_data(path="v1a/train.csv", key_prefix=prefix + "/input/training")
valid_path = sess.upload_data(path="v1a/validation.csv", key_prefix=prefix + "/input/validation")
test_path = sess.upload_data(path="v1a/test.csv", key_prefix=prefix + "/input/test")

### Instantiating the estimator

In [9]:
from sagemaker.estimator import Estimator

linear_estimator = Estimator(
    linear_learner_container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.large',
    output_path='s3://{}/{}/output'.format(bucket,prefix)
)
# setting mini_batch_size to 100 since my dataset is large
linear_estimator.set_hyperparameters(predictor_type='binary_classifier', mini_batch_size=100)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [11]:
training_data_channel = sagemaker.TrainingInput(s3_data=train_path,content_type='text/csv')
validation_data_channel = sagemaker.TrainingInput(s3_data=valid_path,content_type='text/csv')

# Training Phase
### Fitting the model

In [ ]:
linear_estimator.fit({'train': training_data_channel,'validation': validation_data_channel})

INFO:sagemaker:Creating training-job with name: linear-learner-2023-11-02-19-51-56-299


2023-11-02 19:51:56 Starting - Starting the training job...
2023-11-02 19:52:11 Starting - Preparing the instances for training......
2023-11-02 19:53:00 Downloading - Downloading input data...
2023-11-02 19:53:55 Training - Downloading the training image......
2023-11-02 19:54:56 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[11/02/2023 19:55:00 INFO 140138095695680] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': '

# Batch Transform Deployment

In [ ]:
transformer = linear_estimator.transformer(instance_count=1, instance_type="ml.m4.xlarge", assemble_with="Line", output_path=f"s3://{bucket}/{prefix}/batch_output")

*We have to redo this since the csv is saved in a column name row manner instead of index row manner*

In [ ]:
test_df = pd.read_csv("v1a/test.csv")
test_batch = test_df[test_df.columns[1:]]

### Saving the data to current instance of sagemaker

In [ ]:
test_batch.to_csv("data/batch/batch_input_v1a.csv", index = False, header = False)

### Uploading the batch test data to S3

In [ ]:
batch_test_path = sess.upload_data(path="data/batch/batch_input_v1a.csv", key_prefix=prefix + "/batch_input")

### Transforming the data

In [44]:
transformer.transform(batch_test_path, content_type = "text/csv", split_type="Line")
transformer.wait()

INFO:sagemaker:Creating transform job with name: linear-learner-2023-11-02-15-42-18-479


......................................Docker entrypoint called with argument(s): serve
Running default environment configuration script
[11/02/2023 15:48:40 INFO 139872619841344] Memory profiler is not enabled by the environment variable ENABLE_PROFILER.
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:495: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if cons['type'] is 'ineq':
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:743: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(self.X_min) is not 0:
[11/02/2023 15:48:44 WARNING 139872619841344] Loggers have already been setup.
[11/02/2023 15:48:44 INFO 139872619841344] loaded entry point class algorithm.serve.server_config:config_api
[11/02/2023 15:48:44 INFO 139872619841344] loading entry points
[11/02/2023 15:48:4

# Testing

In [48]:
import io
y_file = boto3.client("s3").get_object(Bucket = bucket, Key = f"{prefix}/batch_output/batch_input.csv.out")
y_pred = pd.read_csv(io.BytesIO(y_file["Body"].read()), header = None, names = ["Predicted"])

In [59]:
y_pred["target"] = y_pred.index

### Checking Predicted Values

In [60]:
y_pred

,Predicted,target
"{""predicted_label"":0",score:0.164094552397727},"{""predicted_label"":0"
"{""predicted_label"":0",score:0.215210318565368},"{""predicted_label"":0"
"{""predicted_label"":0",score:0.303584367036819},"{""predicted_label"":0"
"{""predicted_label"":0",score:0.233742132782936},"{""predicted_label"":0"
"{""predicted_label"":0",score:0.12163545936346},"{""predicted_label"":0"
...,...,...
"{""predicted_label"":0",score:0.130352571606636},"{""predicted_label"":0"
"{""predicted_label"":0",score:0.258050173521041},"{""predicted_label"":0"
"{""predicted_label"":0",score:0.045581378042697},"{""predicted_label"":0"
"{""predicted_label"":0",score:0.357890784740448},"{""predicted_label"":0"


## Model Evaluation

In [66]:
predicted_values = y_pred['target'].apply(lambda x: 1 if x == 1 else 0)

In [69]:
from sklearn.metrics import accuracy_score

known_labels = test_df.iloc[:, 0]
accuracy = accuracy_score(known_labels, predicted_values)
print("Accuracy:", accuracy)

Accuracy: 0.7913083175048301


# Step 3: Build and evaluate ensembe models

Write code to perform the follwoing steps:
1. Split data into training, validation and testing sets (70% - 15% - 15%).
2. Use xgboost estimator to build a classifcation model.
3. Host the model on another instance
4. Perform batch transform to evaluate the model on testing data
5. Report the performance metrics that you see better test the model performance 
6. write down your observation on the difference between the performance of using the simple and ensemble models.
Note: You are required to perform the above steps on the two combined datasets separatey.